### 7-4．토픽추출을 위한 데이터 준비
#### 전 기사의 형태소 분석

In [1]:
# konlpy의 인스톨
!pip install konlpy

     |████████████████████████████████| 19.4 MB 2.9 MB/s eta 0:00:01
     |██████████████████████████████▏ | 81 kB 8.2 MB/s eta 0:00:011     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 372 kB 30.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.6 MB 5.6 MB/s eta 0:00:01     |██                              | 276 kB 5.6 MB/s eta 0:00:01
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.3
    Uninstalling beautifulsoup4-4.9.3:
      Successfully uninstalled beautifulsoup4-4.9.3


In [2]:
# nltk의 인스톨
!pip install nltk

     |████████████████████████████████| 1.4 MB 1.4 MB/s eta 0:00:01     |███████████▏                    | 501 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.4 MB/s  eta 0:00:01
     |████████████████████████████████| 284 kB 24.2 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434675 sha256=4b7499ff19458f7e09dd2a7a4d8aafb599d3b7433f391c9e28c96eec6abe6906
  Stored in directory: /Users/hawk90/Library/Caches/pip/wheels/de/5e/42/64abaeca668161c3e2cecc24f864a8fc421e3d07a104fc8a51
Successfully built nltk


In [3]:
import os
import re
from konlpy.tag import Okt
okt = Okt()

# 기사 폴더의 지정
dirs = ['it-life-hack', 'movie-enter']

# 단어와 레이블을 보존하는 리스트를 생성
docterm = [] # 설명변수
label = [] # 목적변수
tmp1 = []
tmp2 = ''

# 각 폴더의 파일을 하나씩 읽어들여서 표시
for i, d in enumerate(dirs):
    # 파일의 취득
    files = os.listdir('./data/' + d)
    
    for file in files:
        # 파일 오픈과 리드
        f = open('./data/' + d + '/' + file, 'r', encoding='utf-8')
        text = f.read()
        
        # 정규표현으로 불필요한 문자열을 제거하고 표시
        reg_text = re.sub(r'[0-9a-zA-Z]+', '', text)
        reg_text = re.sub(r'[:;/+\.-]', '', reg_text)
        reg_text = re.sub(r'[\s\n]', '', reg_text)
        reg_text = reg_text.replace('\n','')        
        
        # 필터로 형태소 분석
        for token in okt.nouns(reg_text):
            tmp1.append(token)
            tmp2 = ' '.join(tmp1)
        # 기사별로 단어를 보존
        docterm.append(tmp2)
        tmp1 = []
        
        # 기사별로 레이블을 보존
        label.append(i)
        
        # 파일 크로즈
        f.close()

JVMNotFoundException: No JVM shared library file (libjli.dylib) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

# 단어의 TF-IDF를 계산
tv = TfidfVectorizer(min_df=0.05, max_df=0.5)
docterm_tv = tv.fit_transform(np.array(docterm))
docterm_tfidf = docterm_tv.toarray()

# 데이터 프레임 형식으로 선두5행까지 표시
docterm_tfidf = pd.DataFrame(docterm_tfidf)
docterm_tfidf.head()

In [ ]:
# 데이터 세트에 레이블열을 결합
label = pd.DataFrame(label)
label = label.rename(columns={0:'label'})

docterm_df = pd.concat([docterm_tfidf, label], axis=1)
docterm_df.head()

#### 유사도의 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# label이 0인 단어문서행렬만 선택
docterm_0 = docterm_df[docterm_df['label'] == 0]
docterm_0 = docterm_0.drop('label', axis=1)

# 단어 페어의 코사인 유사도를 계산
sim0 = cosine_similarity(docterm_0.T)
sim0_df = pd.DataFrame(sim0)

# 유사도를 행렬형식으로 확인
sim0_df

#### 동시출현어 리스트의 작성

In [ ]:
# 유사도 행렬을 리스트 형식으로 변환
sim0_stack = sim0_df.stack()

# 단어 인덱스와 유사도를 분리
index = pd.Series(sim0_stack.index.values)
value = pd.Series(sim0_stack.values)

print(index.head())
print(value.head())

In [ ]:
tmp3 = []
tmp4 = []
for i in range(len(index)):
    # 유사도가 0.5이상 0.9이하인 단어 페어를 추출
    if value[i] >=0.5 and value[i] <= 0.9:
        # 단어 페어를 추출
        tmp1 = str(index[i][0]) + ' ' + str(index[i][1])
        tmp2 = [int(s) for s in tmp1.split()]
        tmp3.append(tmp2)
        # 유사도를 추출
        tmp4 = np.append(tmp4, value[i])

# 동시 출현어 리스트를 작성
tmp3 = pd.DataFrame(tmp3)
tmp3 = tmp3.rename(columns={0:'node1', 1:'node2'})
tmp4 = pd.DataFrame(tmp4)
tmp4 = tmp4.rename(columns={0:'weight'})
sim0_list = pd.concat([tmp3, tmp4], axis=1)

# 작성한 리스트를 확인
sim0_list.head()

In [ ]:
# 연습문제5
# label이 1인 단어 문서행렬만 선택
docterm_1 = docterm_df[docterm_df['label'] == 1]
docterm_1 = docterm_1.drop('label', axis=1)

# 단어 페어의 코사인 유사도를 계산
sim1 = cosine_similarity(docterm_1.T)
sim1_df = pd.DataFrame(sim1)

# 유사도를 행렬형식으로 확인
sim1_df

In [ ]:
# 연습문제6
# 유사도 행렬을 리스트형식으로 변환
sim1_stack = sim1_df.stack()

# 단어 인덱스와 유사도를 분리
index = pd.Series(sim1_stack.index.values)
value = pd.Series(sim1_stack.values)

print(index.head())
print(value.head())

In [ ]:
# 연습문제 6
tmp3 = []
tmp4 = []
for i in range(len(index)):
    # 유사도가 0.5이상 0.9이하인 단어 페어를 추출
    if value[i] >=0.5 and value[i] <= 0.9:
        # 단어 페어를 추출
        tmp1 = str(index[i][0]) + ' ' + str(index[i][1])
        tmp2 = [int(s) for s in tmp1.split()]
        tmp3.append(tmp2)
        # 유사도를 추출
        tmp4 = np.append(tmp4, value[i])

# 동시 출현어 리스트를 작성
tmp3 = pd.DataFrame(tmp3)
tmp3 = tmp3.rename(columns={0:'node1', 1:'node2'})
tmp4 = pd.DataFrame(tmp4)
tmp4 = tmp4.rename(columns={0:'weight'})
sim1_list = pd.concat([tmp3, tmp4], axis=1)

# 작성한 리스트를 확인
sim1_list.head()